In [1]:
# Array and image processing toolboxes
import numpy as np 
import skimage
import skimage.io
import skimage.color
import skimage.transform

In [2]:
DIMX  = 256
DIMY  = 256
DIMZ  = 256

In [3]:
# Read the volume
fname = 'data/image_3d/bucky_256.tif'
image = skimage.io.imread(fname)
image = np.transpose(image, [1, 2, 0])

print(image.shape)

##### Doing projection
alpha_path = None
if alpha_path==None: 
    # Generate random alpha value
    # lut = np.random.uniform(low=0, high=256, size=256).astype(np.uint8)
    lut = np.linspace(start=0, stop=10, num=256, endpoint=False).astype(np.uint8)
    #lut = 32.0 * np.ones_like(lut)
    # lut[0] = 0.0
    # lut[0] = 0.1
    # lut[1] = 0.6
    # ..
    # lut[255] = 0.2
else:
    pass

(256, 256, 256)


In [4]:
color_s = image.copy() 					# Construct the per-voxel color (or resample _s)
alpha_s = lut[color_s.astype(np.uint8)].astype(np.uint8)	# Construct the per-voxel alpha (or resample _s)

color_h = np.zeros_like(image).astype(np.float32)
alpha_h = np.zeros_like(image).astype(np.float32)


color = np.zeros((DIMY, DIMX), dtype=np.float32)
alpha = np.zeros((DIMY, DIMX), dtype=np.float32)

isBackToFront = True
if isBackToFront:
    # Back-to-front compositing
    for i in range(DIMZ-1, -1, -1): # March from 0 to 255
        ci = color_s[:,:,i]/255.0
        ai = alpha_s[:,:,i]/255.0
        # ch_i = ci + (1-ai)*ch_ip1
        # ah_i = ai + (1-ai)*ah_ip1
        if i==DIMZ-1: # the backest slide
            color_h[:,:,i] = ci
            alpha_h[:,:,i] = ai
        else:
            color_h[:,:,i] = ci + (1-ai)*color_h[:,:,i+1]
            alpha_h[:,:,i] = ai + (1-ai)*alpha_h[:,:,i+1]
            #color_h[:,:,i] = ci + (1-alpha_h[:,:,i])*color_h[:,:,i+1]
            #alpha_h[:,:,i] = ai + (1-alpha_h[:,:,i])*alpha_h[:,:,i+1]
            #color_h[:,:,i] = ci + (1-alpha_h[:,:,i+1])*color_h[:,:,i+1]
            #alpha_h[:,:,i] = ai + (1-alpha_h[:,:,i+1])*alpha_h[:,:,i+1]
else: 
    pass
#     # Front-to-back compositing
#     for i in range(0, 256, 1): # March from 255 to 0
#         ci = color_s[:,:,i]/255.0
#         ai = alpha_s[:,:,i]/255.0
#         # ch_i = (1-ahim1)*ci + ch_im1
#         # ah_i = (1-ahim1)*ai + ah_im1
#         if i==0: # the backest slide
#             color_h[:,:,i] = ci
#             alpha_h[:,:,i] = ai
#         else:
#             color_h[:,:,i] = (1-alpha_h[:,:,i-1])*ci + color_h[:,:,i-1]
#             alpha_h[:,:,i] = (1-alpha_h[:,:,i-1])*ai + alpha_h[:,:,i-1]
            
color_h = np.transpose(color_h, [2, 0, 1])
color_s = np.transpose(color_s, [2, 0, 1])
color_h = 255*color_h
skimage.io.imsave("color_h.tif", np.clip(color_h, 0, 255).astype(np.uint8))
skimage.io.imsave("color_s.tif", np.clip(color_s, 0, 255).astype(np.uint8))
skimage.io.imsave("alpha_s.tif", np.clip(alpha_s, 0, 255).astype(np.uint8))

# Create the img2d image
img2d = np.zeros((DIMY, DIMX, 3), dtype=np.float32)
color = skimage.color.gray2rgb(color_h[:,:,0])
img2d = color.copy()
skimage.io.imsave("img2d.tif", np.clip(img2d, 0, 255).astype(np.uint8))

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: alpha_s.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
